In [1]:
from sdRDM.generator import generate_python_api

In [2]:
# generate_python_api("specifications/enzymepynetics.md", "", "EnzymePynetics")

In [4]:
%reload_ext autoreload
%autoreload 2

from EnzymePynetics import Estimator

## Initialize the Estimator from an EnzymeML document

In [5]:
e = Estimator.from_enzymeml(
    enzymeml_doc="/Users/max/Documents/GitHub/MTPHandler/SLAC_kinetic_assay.json",
    measured_reactant="ABTS",
)

abts, abts_radical = e.reactants
slac = e.enzymes[0]

## Define the reaction

In [6]:
oxidation = e.add_reaction(
    id="r1", name="Oxidation", educt=abts, product=abts_radical, enzyme=slac
)

In [7]:
e._init_km

100.0

## Add a kinetic model

In [8]:
michaelis = e.add_model(
    id="model1",
    name="michaelis-menten",
    equation="substrate = -substrate * enzyme * k_cat / (K_m + substrate)",
)

enzyme_inactivation = e.add_model(
    id="model2",
    name="enzyme inactivation",
    equation="enzyme = -enzyme * k_ie",
)

competitive_product = e.add_model(
    id="model3",
    name="competitive product inhibition",
    equation="substrate = -substrate * enzyme * k_cat / K_m * (1 + product / K_ic) + substrate",
)

enzyme_constant = e.add_model(
    id="model4",
    name="constant enzyme",
    equation="enzyme = enzyme",
)

In [ ]:
e._combine_models()

AttributeError: 'Estimator' object has no attribute '_combine_models'

## Next steps:

- check unit consistency
- model callable
- model combinations
- fitting
- visualization

In [ ]:
print(competitive_product)

KineticModel
├── id = model3
├── name = Competitive Product Inhibition
├── equation = substrate = -substrate * enzyme * k_cat / K_m * (1 + product / K_ic) + substrate
└── parameters
    ├── 0
    │   └── KineticParameter
    │       ├── id = kineticparameter30
    │       ├── name = K_m
    │       ├── value = nan
    │       ├── unit = umol / l
    │       ├── initial_value = 100.0
    │       ├── upper = 10000.0
    │       ├── lower = 0.1
    │       ├── is_global = False
    │       ├── constant = False
    │       └── ontology = SBO:0000027
    ├── 1
    │   └── KineticParameter
    │       ├── id = kineticparameter31
    │       ├── name = k_cat
    │       ├── value = nan
    │       ├── unit = 1 / min
    │       ├── initial_value = 8.756705705198392
    │       ├── upper = 875.6705705198392
    │       ├── lower = 0.008756705705198393
    │       ├── is_global = False
    │       ├── constant = False
    │       └── ontology = SBO:0000025
    └── 2
        └── KineticParameter

Protein(id='p0', name='SLAC', vessel_id='plate0', init_conc=None, constant=True, unit=None, uri=None, creator_id=None, sequence='MSSKSKPKDVKV', ecnumber=None, organism=None, organism_tax_id=None, uniprotid=None, ontology='SBO:0000013')

In [ ]:
from scipy.integrate import odeint
from lmfit import Model, Parameters

In [ ]:
m = Model(michaelis.function)
m.param_names
m.independent_vars

['enzyme']

In [ ]:
moodels = [michaelis, enzyme_inactivation]

In [ ]:
lmfit_params = Parameters()

for model in moodels:
    for param in model.parameters:
        lmfit_params.add(
            name=param.name, value=param.initial_value, min=param.lower, max=param.upper
        )

In [ ]:
michaelis.function.__code__.co_varnames

('enzyme', 'K_m', 'substrate', 'k_cat')

In [ ]:
lmfit_params

name,value,initial value,min,max,vary
k_cat,8.75670571,8.756705705198392,0.00875671,875.670571,True
K_m,100.000000,100.0,0.10000000,10000.0000,True
k_ie,0.00770164,0.007701635339554948,7.7016e-06,0.77016353,True


In [ ]:
enzyme_eq = enzyme_inactivation.function
substrate_eq = michaelis.function

In [ ]:
def DS(
    substrate,
    enzyme,
    k_cat,
    K_m,
):
    return -substrate * enzyme * k_cat / (K_m + substrate)


def DE(enzyme, k_ie):
    return -enzyme * k_ie

In [ ]:
def model(
    species,
    time,
    params,
):
    species_dict = dict(zip(["substrate", "enzyme", "product"], species))
    params_dict = params.valuesdict()
    combined_dict = species_dict | params_dict

    subtrate_args = substrate_eq.__code__.co_varnames
    substrate_dict = {k: combined_dict[k] for k in subtrate_args}

    enzyme_args = enzyme_eq.__code__.co_varnames
    enzyme_dict = {k: combined_dict[k] for k in enzyme_args}

    d_substrate = substrate_eq(**substrate_dict)
    d_enzyme = enzyme_eq(**enzyme_dict)
    d_product = -d_substrate

    return [d_substrate, d_enzyme, d_product]

In [ ]:
def alternate_model(species, time, params):
    params = params.valuesdict()

    substrate, enzyme, product = species

    d_substrate = -substrate * enzyme * params["k_cat"] / (params["K_m"] + substrate)
    d_enzyme = -enzyme * params["k_ie"]

    return np.array([d_substrate, d_enzyme, -d_substrate])

In [ ]:
%%timeit
alternate_model(
    [e.substrate_data, e.enzyme_data, e.product_data], e.time_data, lmfit_params
)

1.06 ms ± 4.73 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
%%timeit
model([e.substrate_data, e.enzyme_data, e.product_data], e.time_data, lmfit_params)

1.05 ms ± 5.32 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
def simulate(times, init_conditions, params):
    return np.array(
        [
            odeint(
                func=model,
                y0=init_condition,
                t=time,
                args=(params,),
            )
            for init_condition, time in zip(init_conditions, times)
        ]
    )

In [ ]:
np.array([e.substrate_data[:, 0], e.enzyme_data[:, 0], e.product_data[:, 0]]).T[0]

array([nan,  4., nan])

In [ ]:
%%timeit
r = simulate(
    e.time_data,
    np.array([e.substrate_data[:, 0], e.enzyme_data[:, 0], e.product_data[:, 0]]).T,
    lmfit_params,
)



6.64 ms ± 90.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
import numpy as np
from lmfit import Parameters, minimize
from scipy.integrate import odeint

# Define your ODE model function


def ode_model(params, t, y):
    k1 = params["k1"].value
    k2 = params["k2"].value

    dydt = np.zeros_like(y)
    dydt[0] = -k1 * y[0]
    dydt[1] = k1 * y[0] - k2 * y[1]

    return dydt


# Create a Parameters object and set initial guess values
params = Parameters()
params.add("k1", value=0.1)
params.add("k2", value=0.2)

# Define the time points and experimental data
t = np.array([0.0, 1.0, 2.0, 3.0, 4.0])
data = np.array([1.0, 0.5, 0.3, 0.2, 0.1])

# Define the residual function


def residual(params, t, data):
    y0 = [1.0, 0.0]  # Initial conditions
    model_result = odeint(ode_model, y0, t, args=(params,))
    # Extract the second component of the solution
    model_data = model_result[:, 1]
    return model_data - data


# Perform the fitting
result = minimize(residual, params, args=(t, data))

# Print the fitted parameter values
print("Fitted Parameters:")
print(result.params)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
import numpy as np

# Create a NumPy array with shape (3, 12)
arr = np.array(
    [
        [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
        [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
        [25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36],
    ]
)

# Define a function to calculate row sums


def row_sum(row):
    return np.sum(row)


# Use np.vectorize to apply the row_sum function row-wise (axis=1)
vectorized_row_sum = np.vectorize(row_sum, signature="(n)->()")

# Apply the vectorized function to the entire array row-wise
row_sums = vectorized_row_sum(arr)

# Print the row sums
print(row_sums)

[ 78 222 366]
